<a href="https://colab.research.google.com/github/rahul-art/colabcode/blob/master/tez_imdb_review_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

     |████████████████████████████████| 163kB 20.7MB/s 
     |████████████████████████████████| 17.7MB 192kB/s 
     |████████████████████████████████| 245kB 45.5MB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1


In [ ]:
!pip install tez

In [ ]:
!pip install transformers

     |████████████████████████████████| 1.5MB 13.7MB/s 
     |████████████████████████████████| 890kB 50.2MB/s 
     |████████████████████████████████| 2.9MB 52.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=c4892e9af15645548b64c513232fc73560378b83c3183063e51e6bc64707eafa
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import datasets
train_ds, test_ds = datasets.load_dataset('imdb', split=['train', 'test'])

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/90099cb476936b753383ba2ae6ab2eae419b2e87f71cd5189cb9c8e5814d12a3. Subsequent calls will reuse this data.


In [ ]:
train_ds.features['text']

Value(dtype='string', id=None)

In [ ]:
len(train_ds)

25000

In [ ]:
import torch
import tez
import transformers
import torch.nn as nn
from transformers import  AdamW,get_linear_schedule_with_warmup
from sklearn import metrics
import pandas as pd

In [ ]:

class BERTDataset:
  def __init__(self,texts, targets,max_len=64):
    self.texts=texts
    self.targets=targets
    self.tokenizer=transformers.BertTokenizer.from_pretrained(
        "bert-base-uncased",
        do_lower_case=False
    )
    self.max_len = max_len

# def __len__(self):
 #   return len(self.texts)

  def __getitem__(self,idx):
    text = str(self.texts[idx])
    inputs = self.tokenizer.encode_plus(
        text,
        None,
        add_special_tokens=True,
        max_length=self.max_len,
        padding="max_length",
        truncation=True
        )
    resp = {
        "ids": torch.tensor(inputs['input_ids'],dtype=torch.long),
        "mask": torch.tensor(inputs['mask'],dtype=torch.long),
        "token_type_ids": torch.tensor(inputs['token_type_ids'],dtype=torch.long),
        "targets": torch.tensor(self.targets[idx],dtype=torch.long)
    }
    return resp
-

In [ ]:
class TextModel(tez.Model):
  def __init__(self,num_classes,num_train_steps):
    super().__init__()
    self.bert = transformers.BertModel.from_pretrained("bert-base-uncased",return_dict=False)
    self.bert_drop = nn.Dropout(0.3)
    self.out = nn.Linear(768,num_classes)
    self.num_train_steps = num_train_steps
    self.step_scheduler_after = "batch"
  def fetch_optimizer(self):
    opt = AdamW(self.parameters(),lr=1e-4)
    return opt
  def fetch_scheduler(self):
    sch = get_linear_schedule_with_warmup(
        self.optimizer,
        self.optimizer,num_warmup_steps=0,
        num_training_steps=self.num_train_steps
    )
    return sch

  def loss(self,outputs,targets):
    return nn.BCEWithLogitsLoss()(outputs,targets.view(-1,1))
    #return nn.CrossEntropyLoss()(outputs,targets)

  def monitor_metrics(self,outputs,targets):
    #outputs = torch.argmax(outputs,axis=1).cpu().detach().numpy()
    outputs = torch.sigmoid(outputs).cpu().detach().numpy()>=0.5

    targets = targets.cpu().detach().numpy()
    return {
        "acc": metrics.accuracy_score(targets,outputs)
    }

  def forward(self, ids, mask,toke_type_ids, targets=None):
    _, x = self.bert(ids,attention_mask, token_type_ids=token_type_ids)
    x = self.bert_drop(x)
    x = self.out(x)
    if targets is not None:
      loss = self.loss(x,targets)
      met= self.monitor_metrics(x,targets)
      return x, loss, met
    return x , 0, {}

In [ ]:
def train_model(fold):
  train_dataset = BERTDataset(train_ds.features['text'], train_ds.features['label'])
  valid_dataset = BERTDataset(test_ds.features['text'], test_ds.features['label'])
  n_train_steps = int(len(train_ds)/32*10)
  model = TextModel(num_classes=1,num_train_steps=n_train_steps)
  es = tez.callbacks.EarlyStopping(monitor="valid_loss",patience=3,model_path='model.bin')
  model.fit(train_dataset,valid_dataset=valid_dataset,device="cuda",epochs=10, train_bs=32,callbacks=[es])
  model.load('model.bin',device='cuda')
  preds = model.predict(valid_dataset, device='cuda')
  for p in preds:
    print(p)



In [ ]:
if __name__=="__main__":
  train_model(fold=0)+
  -++
  

TypeError: ignored